In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install -U accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hzcqolwb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hzcqolwb
  Resolved https://github.com/huggingface/transformers to commit 6657fb5fed3f94812ef97dda16e91eee77f03d61
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.42.0.dev0-py3-none-any.whl size=9125128 sha256=55e42163a95287c66cd1a355b23b3c226f69dc225d0567a2897a63f880abab20
  Stored in directory: /tmp/pip-ephem-wheel-cache-3d7h16g7/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully uninstalled transformers-4.41.0


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
id2label = {1:"fraudulent",0:"legitimate"}

In [ ]:
label2id = {"fraudulent":1,"legitimate":0}

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer, pipeline
import pandas as pd
torch.random.manual_seed(0)

model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-uncased",
    torch_dtype="auto",
    trust_remote_code=True,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
data = pd.read_csv('online-fraud-dataset (1).csv')
data['label'] = data['target'].map(lambda x: label2id[str(x).strip()] if pd.notna(x) else x)
data['label']

0       1
1       1
2       0
3       0
4       1
       ..
1135    0
1136    0
1137    1
1138    1
1139    1
Name: label, Length: 1140, dtype: int64

In [ ]:
data.shape

(1140, 3)

In [ ]:
data['label'].value_counts()

label
1    579
0    561
Name: count, dtype: int64

In [ ]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

df_shuffled = shuffle(data, random_state=42)

train_df, valid_df = train_test_split(df_shuffled, test_size=0.1, random_state=42)

# Save the separate DataFrames to CSV files
train_csv_path = './bert-classification-fraud-train.csv'
valid_csv_path = './bert-classification-fraud-valid.csv'

#d.to_csv(train_csv_path, index=False)
train_df.to_csv(train_csv_path, index=False)
valid_df.to_csv(valid_csv_path, index=False)

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

# Example: Load your dataset
data_files = {"train":train_csv_path, "validation":valid_csv_path}
#data_files = {"train":train_csv_path}
dataset = load_dataset("csv", data_files=data_files)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'target', 'label'],
        num_rows: 1026
    })
    validation: Dataset({
        features: ['text', 'target', 'label'],
        num_rows: 114
    })
})

In [ ]:
def preprocess_data(examples):
  text = examples["text"]
  return tokenizer(text, padding="max_length", truncation=True, max_length=200)

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=['text','target'])

Map:   0%|          | 0/1026 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import accuracy_score
import torch

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

In [ ]:
from transformers import TrainingArguments, Trainer
args = TrainingArguments(
        f"bert-finetuned-fraud-veg-kebab",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-05,
        lr_scheduler_type='linear',
        per_device_train_batch_size=2,
        num_train_epochs=10,
        weight_decay=0.03,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
        greater_is_better=False,
        save_total_limit = 2,
        push_to_hub=True
    )

trainer = Trainer(
        model=model,
        args=args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1483: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.461200,0.492009,0.894737
2,0.395400,0.414243,0.894737
3,0.289000,0.487892,0.903509
4,0.189100,0.523610,0.929825
5,0.069800,0.669347,0.903509
6,0.034300,0.547277,0.929825
7,0.003900,0.703320,0.912281
8,0.000600,0.741029,0.912281
9,0.000100,0.660382,0.921053
10,0.000000,0.670638,0.921053


TrainOutput(global_step=5130, training_loss=0.1406817062185137, metrics={'train_runtime': 1071.3544, 'train_samples_per_second': 9.577, 'train_steps_per_second': 4.788, 'total_flos': 1054499776560000.0, 'train_loss': 0.1406817062185137, 'epoch': 10.0})